In [ ]:
# Using earthaccess access pattern for direct access "streaming" of cloud-hosted 
# MEaSUREs Phase-Based Antarctica Ice Velocity Map, Version 1
# 
# Code written to run on CryoCloud cloud-computing JupyterHub
# Learn more: https://cryointhecloud.com/
# 
# Written 2023-11-15 by Wilson Sauthoff (wsauthoff.github.io)

In [ ]:
# Import libraries
import earthaccess
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1 import make_axes_locatable
import os
import xarray as xr

In [ ]:
# Log into NASA Earthdata to search for datasets
earthaccess.login()

In [ ]:
# Find cloud-hosted MEaSUREs Phase-Based Antarctica Ice Velocity Map, Version 1
# DOI from https://nsidc.org/data/NSIDC-0754/versions/1
results = earthaccess.search_data(
    doi='10.5067/PZ3NJ5RXRH10',
    cloud_hosted=True,
    bounding_box=(1, -89, -1, -89)  # (lower_left_lon, lower_left_lat , upper_right_lon, upper_right_lat))
)

In [ ]:
# Open data granules as s3 files to stream
files = earthaccess.open(results)
files

In [ ]:
# Print file name to ensure expected dataset
print(files[0])

In [ ]:
# Open each file, which are quadrants in polar stereographic coordinations around the Geographic South Pole
ice_vel = xr.open_dataset(files[0])
ice_vel

In [ ]:
# Specify the variables to keep
variables_to_keep = ['x', 'y', 'VX', 'VY']

variables_to_drop = [var for var in ice_vel.variables if var not in variables_to_keep]

# Drop variables to reduce memory consumption
ice_vel = ice_vel.drop_vars(variables_to_drop)
ice_vel

In [ ]:
# Calculate velocity magnitude
vel_mag = (ice_vel['VX']**2 + ice_vel['VY']**2)**0.5

In [ ]:
# Follow example code to check calulated velocity seems reasonable 
# Test three drill sites from WISSARD and SALSA
coords=[[-295743.804392, -502137.677718],
       [-278502.862779,-561384.761658],
       [-173785.999994,-590925.999998]]

# Calculate the vector magnitude at the sample points and print the result
for site in range(len(coords)):
       print(vel_mag.sel(x=coords[site][0], y=coords[site][1], method="nearest").values, 'm/yr')

In [ ]:
# Plot velocity at continental scale
fig, ax = plt.subplots(figsize=(5,5))
mappable = ax.imshow(vel_mag,
    extent=[vel_mag['x'].min()/1000, vel_mag['x'].max()/1000,
            vel_mag['y'].min()/1000, vel_mag['y'].max()/1000],
    norm=LogNorm(), cmap='magma')
ax.set_xlabel('x [km]')
ax.set_ylabel('y [km]')
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.2)
cbar = fig.colorbar(mappable, cax=cax)
cbar.set_label('velocity [m/yr]')
plt.show()